## Definition of genres

In [6]:
import numpy as np
import librosa as lbr
import tensorflow.keras.backend as K

In [7]:
GENRES = ['Electronic', 'Experimental', 'Folk', 'HipHop', 'Instrumental', 'International', 'Pop','Rock']
WINDOW_SIZE = 2048
WINDOW_STRIDE = WINDOW_SIZE // 2
N_MELS = 128
MEL_KWARGS = {
    'n_fft': WINDOW_SIZE,
    'hop_length': WINDOW_STRIDE,
    'n_mels': N_MELS
}


## Feature Engineering

In [28]:
def get_layer_output_function(model, layer_name):
    input = model.get_layer('input').input
    output = model.get_layer(layer_name).output
    f = K.function([input, K.learning_phase()], [output])
    return lambda x: f([x, 0])[0] # learning_phase = 0 means test

def load_track(filename, enforce_shape=None):
    new_input, sample_rate = lbr.load(filename, mono=True, duration=40.0)
    features = lbr.feature.melspectrogram(new_input, **MEL_KWARGS).T

    if enforce_shape is not None:
        if features.shape[0] < enforce_shape[0]:
            delta_shape = (enforce_shape[0] - features.shape[0],
                    enforce_shape[1])
            features = np.append(features, np.zeros(delta_shape), axis=0)
        elif features.shape[0] > enforce_shape[0]:
            features = features[: enforce_shape[0], :]

    features[features == 0] = 1e-6
    return (np.log(features), float(new_input.shape[0]) / sample_rate)


## Creating Data Pickle

In [16]:
import sys
import numpy as np
from math import pi
from pickle import dump
import os
from optparse import OptionParser

In [17]:
TRACK_COUNT = 10000

In [18]:
def get_default_shape(dataset_path):
    tmp_features, _ = load_track(os.path.join(dataset_path,
        'Electronic/Electronic.00000.mp3'))
    return tmp_features.shape

def collect_data(dataset_path):
    '''
    :param dataset_path: path to the dataset directory
    :returns: triple (x, y, track_paths) where x is a matrix containing
        extracted features, y is a one-hot matrix of genre labels and
        track_paths is a dict of absolute track paths indexed by row indices in
        the x and y matrices
    '''
    default_shape = get_default_shape(dataset_path)
    x = np.zeros((TRACK_COUNT,) + default_shape, dtype=np.float32)
    y = np.zeros((TRACK_COUNT, len(GENRES)), dtype=np.float32)
    track_paths = {}

    for (genre_index, genre_name) in enumerate(GENRES):
        for i in range(1000):            
            file_name = '{}/{}.{}.mp3'.format(genre_name,
                    genre_name, '%05d' % i)
            print('Processing', file_name)
            
            path = os.path.join(dataset_path, file_name)
            print(genre_index,i)
            track_index = genre_index  * 1000 + i
            
            x[track_index], _ = load_track(path, default_shape)
            y[track_index, genre_index] = 1
            track_paths[track_index] = os.path.abspath(path)

    return (x, y, track_paths)

In [19]:
dataset_path=os.path.join('/Users/jairomelo/Desktop/ML/YORK/ML1020/Final Project/genres')
output_pkl_path=os.path.join('/Users/jairomelo/Desktop/ML/YORK/ML1020/Final Project/data.pkl')


## Creating Data File

In [ ]:
(x, y, track_paths) = collect_data(dataset_path)

Processing Electronic/Electronic.00000.mp3
0 0
Processing Electronic/Electronic.00001.mp3
0 1
Processing Electronic/Electronic.00002.mp3
0 2
Processing Electronic/Electronic.00003.mp3
0 3
Processing Electronic/Electronic.00004.mp3
0 4
Processing Electronic/Electronic.00005.mp3
0 5
Processing Electronic/Electronic.00006.mp3
0 6
Processing Electronic/Electronic.00007.mp3
0 7
Processing Electronic/Electronic.00008.mp3
0 8
Processing Electronic/Electronic.00009.mp3
0 9
Processing Electronic/Electronic.00010.mp3
0 10
Processing Electronic/Electronic.00011.mp3
0 11
Processing Electronic/Electronic.00012.mp3
0 12
Processing Electronic/Electronic.00013.mp3
0 13
Processing Electronic/Electronic.00014.mp3
0 14
Processing Electronic/Electronic.00015.mp3
0 15
Processing Electronic/Electronic.00016.mp3
0 16
Processing Electronic/Electronic.00017.mp3
0 17
Processing Electronic/Electronic.00018.mp3
0 18
Processing Electronic/Electronic.00019.mp3
0 19
Processing Electronic/Electronic.00020.mp3
0 20
Pr

Processing Electronic/Electronic.00170.mp3
0 170
Processing Electronic/Electronic.00171.mp3
0 171
Processing Electronic/Electronic.00172.mp3
0 172
Processing Electronic/Electronic.00173.mp3
0 173
Processing Electronic/Electronic.00174.mp3
0 174
Processing Electronic/Electronic.00175.mp3
0 175
Processing Electronic/Electronic.00176.mp3
0 176
Processing Electronic/Electronic.00177.mp3
0 177
Processing Electronic/Electronic.00178.mp3
0 178
Processing Electronic/Electronic.00179.mp3
0 179
Processing Electronic/Electronic.00180.mp3
0 180
Processing Electronic/Electronic.00181.mp3
0 181
Processing Electronic/Electronic.00182.mp3
0 182
Processing Electronic/Electronic.00183.mp3
0 183
Processing Electronic/Electronic.00184.mp3
0 184
Processing Electronic/Electronic.00185.mp3
0 185
Processing Electronic/Electronic.00186.mp3
0 186
Processing Electronic/Electronic.00187.mp3
0 187
Processing Electronic/Electronic.00188.mp3
0 188
Processing Electronic/Electronic.00189.mp3
0 189
Processing Electroni

Processing Electronic/Electronic.00338.mp3
0 338
Processing Electronic/Electronic.00339.mp3
0 339
Processing Electronic/Electronic.00340.mp3
0 340
Processing Electronic/Electronic.00341.mp3
0 341
Processing Electronic/Electronic.00342.mp3
0 342
Processing Electronic/Electronic.00343.mp3
0 343
Processing Electronic/Electronic.00344.mp3
0 344
Processing Electronic/Electronic.00345.mp3
0 345
Processing Electronic/Electronic.00346.mp3
0 346
Processing Electronic/Electronic.00347.mp3
0 347
Processing Electronic/Electronic.00348.mp3
0 348
Processing Electronic/Electronic.00349.mp3
0 349
Processing Electronic/Electronic.00350.mp3
0 350
Processing Electronic/Electronic.00351.mp3
0 351
Processing Electronic/Electronic.00352.mp3
0 352
Processing Electronic/Electronic.00353.mp3
0 353
Processing Electronic/Electronic.00354.mp3
0 354
Processing Electronic/Electronic.00355.mp3
0 355
Processing Electronic/Electronic.00356.mp3
0 356
Processing Electronic/Electronic.00357.mp3
0 357
Processing Electroni

Processing Electronic/Electronic.00506.mp3
0 506
Processing Electronic/Electronic.00507.mp3
0 507
Processing Electronic/Electronic.00508.mp3
0 508
Processing Electronic/Electronic.00509.mp3
0 509
Processing Electronic/Electronic.00510.mp3
0 510
Processing Electronic/Electronic.00511.mp3
0 511
Processing Electronic/Electronic.00512.mp3
0 512
Processing Electronic/Electronic.00513.mp3
0 513
Processing Electronic/Electronic.00514.mp3
0 514
Processing Electronic/Electronic.00515.mp3
0 515
Processing Electronic/Electronic.00516.mp3
0 516
Processing Electronic/Electronic.00517.mp3
0 517
Processing Electronic/Electronic.00518.mp3
0 518
Processing Electronic/Electronic.00519.mp3
0 519
Processing Electronic/Electronic.00520.mp3
0 520
Processing Electronic/Electronic.00521.mp3
0 521
Processing Electronic/Electronic.00522.mp3
0 522
Processing Electronic/Electronic.00523.mp3
0 523
Processing Electronic/Electronic.00524.mp3
0 524
Processing Electronic/Electronic.00525.mp3
0 525
Processing Electroni

Processing Electronic/Electronic.00674.mp3
0 674
Processing Electronic/Electronic.00675.mp3
0 675
Processing Electronic/Electronic.00676.mp3
0 676
Processing Electronic/Electronic.00677.mp3
0 677
Processing Electronic/Electronic.00678.mp3
0 678
Processing Electronic/Electronic.00679.mp3
0 679
Processing Electronic/Electronic.00680.mp3
0 680
Processing Electronic/Electronic.00681.mp3
0 681
Processing Electronic/Electronic.00682.mp3
0 682
Processing Electronic/Electronic.00683.mp3
0 683
Processing Electronic/Electronic.00684.mp3
0 684
Processing Electronic/Electronic.00685.mp3
0 685
Processing Electronic/Electronic.00686.mp3
0 686
Processing Electronic/Electronic.00687.mp3
0 687
Processing Electronic/Electronic.00688.mp3
0 688
Processing Electronic/Electronic.00689.mp3
0 689
Processing Electronic/Electronic.00690.mp3
0 690
Processing Electronic/Electronic.00691.mp3
0 691
Processing Electronic/Electronic.00692.mp3
0 692
Processing Electronic/Electronic.00693.mp3
0 693
Processing Electroni

...

Processing Folk/Folk.00140.mp3
2 140
Processing Folk/Folk.00141.mp3
2 141
Processing Folk/Folk.00142.mp3
2 142
Processing Folk/Folk.00143.mp3
2 143
Processing Folk/Folk.00144.mp3
2 144
Processing Folk/Folk.00145.mp3
2 145
Processing Folk/Folk.00146.mp3
2 146
Processing Folk/Folk.00147.mp3
2 147
Processing Folk/Folk.00148.mp3
2 148
Processing Folk/Folk.00149.mp3
2 149
Processing Folk/Folk.00150.mp3
2 150
Processing Folk/Folk.00151.mp3
2 151
Processing Folk/Folk.00152.mp3
2 152
Processing Folk/Folk.00153.mp3
2 153
Processing Folk/Folk.00154.mp3
2 154
Processing Folk/Folk.00155.mp3
2 155
Processing Folk/Folk.00156.mp3
2 156
Processing Folk/Folk.00157.mp3
2 157
Processing Folk/Folk.00158.mp3
2 158
Processing Folk/Folk.00159.mp3
2 159
Processing Folk/Folk.00160.mp3
2 160
Processing Folk/Folk.00161.mp3
2 161
Processing Folk/Folk.00162.mp3
2 162
Processing Folk/Folk.00163.mp3
2 163
Processing Folk/Folk.00164.mp3
2 164
Processing Folk/Folk.00165.mp3
2 165
Processing Folk/Folk.00166.mp3
2 166
P

Processing Folk/Folk.00362.mp3
2 362
Processing Folk/Folk.00363.mp3
2 363
Processing Folk/Folk.00364.mp3
2 364
Processing Folk/Folk.00365.mp3
2 365
Processing Folk/Folk.00366.mp3
2 366
Processing Folk/Folk.00367.mp3
2 367
Processing Folk/Folk.00368.mp3
2 368
Processing Folk/Folk.00369.mp3
2 369
Processing Folk/Folk.00370.mp3
2 370
Processing Folk/Folk.00371.mp3
2 371
Processing Folk/Folk.00372.mp3
2 372
Processing Folk/Folk.00373.mp3
2 373
Processing Folk/Folk.00374.mp3
2 374
Processing Folk/Folk.00375.mp3
2 375
Processing Folk/Folk.00376.mp3
2 376
Processing Folk/Folk.00377.mp3
2 377
Processing Folk/Folk.00378.mp3
2 378
Processing Folk/Folk.00379.mp3
2 379
Processing Folk/Folk.00380.mp3
2 380
Processing Folk/Folk.00381.mp3
2 381
Processing Folk/Folk.00382.mp3
2 382
Processing Folk/Folk.00383.mp3
2 383
Processing Folk/Folk.00384.mp3
2 384
Processing Folk/Folk.00385.mp3
2 385
Processing Folk/Folk.00386.mp3
2 386
Processing Folk/Folk.00387.mp3
2 387
Processing Folk/Folk.00388.mp3
2 388
P

Processing Folk/Folk.00584.mp3
2 584
Processing Folk/Folk.00585.mp3
2 585
Processing Folk/Folk.00586.mp3
2 586
Processing Folk/Folk.00587.mp3
2 587
Processing Folk/Folk.00588.mp3
2 588
Processing Folk/Folk.00589.mp3
2 589
Processing Folk/Folk.00590.mp3
2 590
Processing Folk/Folk.00591.mp3
2 591
Processing Folk/Folk.00592.mp3
2 592
Processing Folk/Folk.00593.mp3
2 593
Processing Folk/Folk.00594.mp3
2 594
Processing Folk/Folk.00595.mp3
2 595
Processing Folk/Folk.00596.mp3
2 596
Processing Folk/Folk.00597.mp3
2 597
Processing Folk/Folk.00598.mp3
2 598
Processing Folk/Folk.00599.mp3
2 599
Processing Folk/Folk.00600.mp3
2 600
Processing Folk/Folk.00601.mp3
2 601
Processing Folk/Folk.00602.mp3
2 602
Processing Folk/Folk.00603.mp3
2 603
Processing Folk/Folk.00604.mp3
2 604
Processing Folk/Folk.00605.mp3
2 605
Processing Folk/Folk.00606.mp3
2 606
Processing Folk/Folk.00607.mp3
2 607
Processing Folk/Folk.00608.mp3
2 608
Processing Folk/Folk.00609.mp3
2 609
Processing Folk/Folk.00610.mp3
2 610
P

Processing Folk/Folk.00806.mp3
2 806
Processing Folk/Folk.00807.mp3
2 807
Processing Folk/Folk.00808.mp3
2 808
Processing Folk/Folk.00809.mp3
2 809
Processing Folk/Folk.00810.mp3
2 810
Processing Folk/Folk.00811.mp3
2 811
Processing Folk/Folk.00812.mp3
2 812
Processing Folk/Folk.00813.mp3
2 813
Processing Folk/Folk.00814.mp3
2 814
Processing Folk/Folk.00815.mp3
2 815
Processing Folk/Folk.00816.mp3
2 816
Processing Folk/Folk.00817.mp3
2 817
Processing Folk/Folk.00818.mp3
2 818
Processing Folk/Folk.00819.mp3
2 819
Processing Folk/Folk.00820.mp3
2 820
Processing Folk/Folk.00821.mp3
2 821
Processing Folk/Folk.00822.mp3
2 822
Processing Folk/Folk.00823.mp3
2 823
Processing Folk/Folk.00824.mp3
2 824
Processing Folk/Folk.00825.mp3
2 825
Processing Folk/Folk.00826.mp3
2 826
Processing Folk/Folk.00827.mp3
2 827
Processing Folk/Folk.00828.mp3
2 828
Processing Folk/Folk.00829.mp3
2 829
Processing Folk/Folk.00830.mp3
2 830
Processing Folk/Folk.00831.mp3
2 831
Processing Folk/Folk.00832.mp3
2 832
P

Processing HipHop/HipHop.00026.mp3
3 26
Processing HipHop/HipHop.00027.mp3
3 27
Processing HipHop/HipHop.00028.mp3
3 28
Processing HipHop/HipHop.00029.mp3
3 29
Processing HipHop/HipHop.00030.mp3
3 30
Processing HipHop/HipHop.00031.mp3
3 31
Processing HipHop/HipHop.00032.mp3
3 32
Processing HipHop/HipHop.00033.mp3
3 33
Processing HipHop/HipHop.00034.mp3
3 34
Processing HipHop/HipHop.00035.mp3
3 35
Processing HipHop/HipHop.00036.mp3
3 36
Processing HipHop/HipHop.00037.mp3
3 37
Processing HipHop/HipHop.00038.mp3
3 38
Processing HipHop/HipHop.00039.mp3
3 39
Processing HipHop/HipHop.00040.mp3
3 40
Processing HipHop/HipHop.00041.mp3
3 41
Processing HipHop/HipHop.00042.mp3
3 42
Processing HipHop/HipHop.00043.mp3
3 43
Processing HipHop/HipHop.00044.mp3
3 44
Processing HipHop/HipHop.00045.mp3
3 45
Processing HipHop/HipHop.00046.mp3
3 46
Processing HipHop/HipHop.00047.mp3
3 47
Processing HipHop/HipHop.00048.mp3
3 48
Processing HipHop/HipHop.00049.mp3
3 49
Processing HipHop/HipHop.00050.mp3
3 50


In [ ]:
##Writing the Pickle/Data - This fails for files greater than 4GB
data = {'x': x, 'y': y, 'track_paths': track_paths}
with open(output_pkl_path, 'wb') as f:
    dump(data, f)

## Training the C RNN Model

In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Lambda, Dropout, Activation, \
        TimeDistributed, Convolution1D, MaxPooling1D, BatchNormalization
from sklearn.model_selection import train_test_split
import numpy as np
import pickle
from optparse import OptionParser
from sys import stderr, argv
import os



In [24]:
SEED = 42
N_LAYERS = 3
FILTER_LENGTH = 5
CONV_FILTER_COUNT = 256
BATCH_SIZE = 32
EPOCH_COUNT = 85



In [25]:
def train_model(data, model_path):
    x = data['x']
    y = data['y']
    (x_train, x_val, y_train, y_val) = train_test_split(x, y, test_size=0.3,
            random_state=SEED)

    print('Building model...')

    n_features = x_train.shape[2]
    input_shape = (None, n_features)
    model_input = Input(input_shape, name='input')
    layer = model_input
    for i in range(N_LAYERS):
        # second convolutional layer names are used by extract_filters.py
        layer = Convolution1D(
                filters=CONV_FILTER_COUNT,
                kernel_size=FILTER_LENGTH,
                name='convolution_' + str(i + 1)
            )(layer)
        layer = BatchNormalization(momentum=0.9)(layer)
        layer = Activation('relu')(layer)
        layer = MaxPooling1D(2)(layer)
        layer = Dropout(0.5)(layer)

    layer = TimeDistributed(Dense(len(GENRES)))(layer)
    time_distributed_merge_layer = Lambda(
            function=lambda x: K.mean(x, axis=1), 
            output_shape=lambda shape: (shape[0],) + shape[2:],
            name='output_merged'
        )
    layer = time_distributed_merge_layer(layer)
    layer = Activation('softmax', name='output_realtime')(layer)
    model_output = layer
    model = Model(model_input, model_output)
    opt = Adam(lr=0.001)
    model.compile(
            loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy']
        )

    print('Training...')
    model.fit(
        x_train, y_train, batch_size=BATCH_SIZE, nb_epoch=EPOCH_COUNT,
        validation_data=(x_val, y_val), verbose=1, callbacks=[
            ModelCheckpoint(
                model_path, save_best_only=True, monitor='val_acc', verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_acc', factor=0.5, patience=10, min_delta=0.01,
                verbose=1
            )
        ]
    )

    return model

In [26]:
model_path=os.path.join('/Users/jairomelo/Desktop/ML/YORK/ML1020/Final Project/model.h5')
output_pkl_path=os.path.join('/Users/jairomelo/Desktop/ML/YORK/ML1020/Final Project/data.pkl')

In [ ]:
##Loading the Pickle/Data - This fails for files greater than 4GB
with open(output_pkl_path, 'rb') as f:
      data = pickle.load(f)

In [27]:
train_model(data, model_path)

Building model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training...
Train on 7000 samples, validate on 3000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
6976/7000 [============================>.] - ETA: 0s - loss: 1.4008 - acc: 0.4266
Epoch 00001: val_acc improved from -inf to 0.50633, saving model to /Users/jairomelo/Desktop/ML/YORK/ML1020/Final Project/model.h5
7000/7000 [==============================] - 86s 12ms/sample - loss: 1.4004 - acc: 0.4273 - val_loss: 1.4070 - val_acc: 0.5063
Epoch 2/100
6976/7000 [============================>.] - ETA: 0s - loss: 1.2774 - acc: 0.5014
Epoch 00002: val_acc improved from 0.50633 to 0.53567, saving model to /Users/jairomelo/Desktop/ML/YORK/ML1020/Final Project/model.h5
7000/7000 [==============================] - 87s 12ms/sample - loss: 1.2779 - acc: 0.5010 - val_loss: 1.253

6976/7000 [============================>.] - ETA: 0s - loss: 0.7451 - acc: 0.5473
Epoch 00025: val_acc did not improve from 0.61800
7000/7000 [==============================] - 91s 13ms/sample - loss: 0.7446 - acc: 0.5477 - val_loss: 1.0227 - val_acc: 0.4760
Epoch 26/100
6976/7000 [============================>.] - ETA: 0s - loss: 0.7319 - acc: 0.5612
Epoch 00026: val_acc did not improve from 0.61800
7000/7000 [==============================] - 91s 13ms/sample - loss: 0.7318 - acc: 0.5610 - val_loss: 1.0643 - val_acc: 0.4553
Epoch 27/100
6976/7000 [============================>.] - ETA: 0s - loss: 0.7235 - acc: 0.5652
Epoch 00027: val_acc did not improve from 0.61800
7000/7000 [==============================] - 92s 13ms/sample - loss: 0.7231 - acc: 0.5654 - val_loss: 1.0905 - val_acc: 0.4560
Epoch 28/100
6976/7000 [============================>.] - ETA: 0s - loss: 0.7116 - acc: 0.5935
Epoch 00028: val_acc did not improve from 0.61800
7000/7000 [==============================] - 92s 13m

6976/7000 [============================>.] - ETA: 0s - loss: 0.3867 - acc: 0.6951
Epoch 00053: val_acc did not improve from 0.66933
7000/7000 [==============================] - 90s 13ms/sample - loss: 0.3874 - acc: 0.6944 - val_loss: 1.1884 - val_acc: 0.4663
Epoch 54/100
6976/7000 [============================>.] - ETA: 0s - loss: 0.3854 - acc: 0.6974
Epoch 00054: val_acc did not improve from 0.66933
7000/7000 [==============================] - 90s 13ms/sample - loss: 0.3862 - acc: 0.6973 - val_loss: 1.2033 - val_acc: 0.4727
Epoch 55/100
6976/7000 [============================>.] - ETA: 0s - loss: 0.3833 - acc: 0.7096
Epoch 00055: val_acc did not improve from 0.66933
7000/7000 [==============================] - 90s 13ms/sample - loss: 0.3826 - acc: 0.7096 - val_loss: 1.2102 - val_acc: 0.4703
Epoch 56/100
6976/7000 [============================>.] - ETA: 0s - loss: 0.3792 - acc: 0.7037
Epoch 00056: val_acc did not improve from 0.66933
7000/7000 [==============================] - 90s 13m

Epoch 82/100
6976/7000 [============================>.] - ETA: 0s - loss: 0.2901 - acc: 0.7341
Epoch 00082: val_acc did not improve from 0.67067
7000/7000 [==============================] - 90s 13ms/sample - loss: 0.2899 - acc: 0.7339 - val_loss: 1.2780 - val_acc: 0.4720
Epoch 83/100
6976/7000 [============================>.] - ETA: 0s - loss: 0.2948 - acc: 0.7344
Epoch 00083: val_acc did not improve from 0.67067
7000/7000 [==============================] - 92s 13ms/sample - loss: 0.2945 - acc: 0.7347 - val_loss: 1.2718 - val_acc: 0.4767
Epoch 84/100
6976/7000 [============================>.] - ETA: 0s - loss: 0.2958 - acc: 0.7253
Epoch 00084: val_acc did not improve from 0.67067
7000/7000 [==============================] - 92s 13ms/sample - loss: 0.2963 - acc: 0.7251 - val_loss: 1.2820 - val_acc: 0.4703
Epoch 85/100
6976/7000 [============================>.] - ETA: 0s - loss: 0.2877 - acc: 0.7388
Epoch 00085: val_acc did not improve from 0.67067
7000/7000 [============================